# T2.2: PANDAS. Ler excels de alcaldes



<p>Instalación de paquetes necesarios</p>

In [ ]:
%conda install pandas # o  %pip install pandas
%conda install datetime #o %pip install datetime
%conda install unidecode # o %pip install unidecode

<p>Importar librerías</p>

In [2]:
import pandas as pd
from datetime import datetime
from unidecode import unidecode


### Normalización del data frame
<p>Cargo todas las páginas del excel y creo el método para limpiar y formatear los datos, después volvemos a juntar las hojas limpiadas.</p>

In [3]:
excel_entero = pd.read_excel("Alcades_1979_2023.xlsx", na_values='\\N', sheet_name=None)

def limpiar(df): 
    # Quitar los valores NaN
    df = df.dropna(axis=1, how='all') 

    # Quitar las columnas innecesarias
    df = df.iloc[7:]

    # Quitar una columna vacía
    df = df.drop(['Unnamed: 13'], axis=1) 

    # Renombrar  los header
    df.rename(columns={'Unnamed: 3': 'COMUNIDAD AUTONOMA', 'Unnamed: 4': 'PROVINCIA', 'Unnamed: 5': 'MUNICIPIO', 'Unnamed: 6': 'CODIGO INE', 'Unnamed: 7': 'NOMBRE', 'Unnamed: 8': 'LISTA', 'Unnamed: 9': 'FECHA POSESION', 'Unnamed: 10': 'FECHA BAJA'}, inplace=True)
    
    # Quitar los  acentos y convertirlo a minuscula
    df = df.map(lambda texto: unidecode(texto.lower().replace(',', '').replace('/', '').replace('.', '').replace('*', '').replace('-', '')) if isinstance(texto, str) else texto)

    # Eliminar las filas donde el nombre este vacío
    df = df[df['NOMBRE'].notna() & (df['NOMBRE'] != '')]


    # Cambiar el formato de la fecha 
    for fecha_col in ['FECHA POSESION', 'FECHA BAJA']: 
        if fecha_col in df.columns:
            df[fecha_col] = pd.to_datetime(df[fecha_col], errors='coerce',dayfirst=True, format="%d/%m/%Y").dt.strftime('%d/%m/%Y') 

    return df 

hojas_limpiadas = []

# Limpiar las hojas y concatenarlas en un solo Dataframe
for hoja in excel_entero:
    hoja_limpiada = limpiar(excel_entero[hoja])  
    hojas_limpiadas.append(hoja_limpiada)  
    

excel_entero_df = pd.concat(hojas_limpiadas, ignore_index=True)



excel_entero_df.head(50)




,COMUNIDAD AUTONOMA,PROVINCIA,MUNICIPIO,CODIGO INE,NOMBRE,LISTA,FECHA POSESION,FECHA BAJA
0,andalucia,almeria,abla,040010,herrerias herrerias antonio,ucd,19/04/1979,NaN
1,andalucia,almeria,abrucena,040025,martinez lao juan,psoe,19/04/1979,NaN
2,andalucia,almeria,adra,040031,sarmiento posada pedro,otros,19/04/1979,NaN
3,andalucia,almeria,albanchez,040046,martos aybar ramon,ucd,19/04/1979,NaN
4,andalucia,almeria,alboloduy,040059,blanes paniagua mariano,ucd,19/04/1979,NaN
5,andalucia,almeria,albox,040062,miras carrasco jose,ucd,19/04/1979,NaN
6,andalucia,almeria,alcolea,040078,lopez mellado gabriel bernardo,ucd,19/04/1979,NaN
7,andalucia,almeria,alcontar,040084,requena martinez manuel,ucd,19/04/1979,NaN
8,andalucia,almeria,alcudia de monteagud,040097,perez padilla rogelio,ucd,19/04/1979,NaN
9,andalucia,almeria,alhabia,040101,castellon diaz francisco,ucd,19/04/1979,NaN


### Fai algunhas averiguacións sobre os datos:

<p>1. Saber qué alcaldes repetiron mandato e en qué lexislaturas (ollo, poden mudar un pouco os nomes).</p>

<p>aaaa</p>

In [4]:
import pandas as pd

# Agrupar por nombre y municipio, y contar las legislaturas
repetidos = (
    excel_entero_df.groupby(['NOMBRE', 'MUNICIPIO'])
    .size()
    .reset_index(name='FRECUENCIA')
)

# Filtrar alcaldes que repitieron mandato 
alcaldes_repetidos = repetidos[repetidos['FRECUENCIA'] > 1]

# Filtrar el dataframe original para incluir solo alcaldes que repitieron mandato
df_repetidos = excel_entero_df.merge(alcaldes_repetidos, on=['NOMBRE', 'MUNICIPIO'])

# Convertir las fechas de posesión a formato datetime y extraer solo los años
df_repetidos['FECHA POSESION'] = pd.to_datetime(df_repetidos['FECHA POSESION'], errors='coerce', dayfirst=True)
df_repetidos = df_repetidos.dropna(subset=['FECHA POSESION'])  # Eliminar fechas inválidas

# Agrupar por nombre y municipio, y extraer las legislaturas (años) únicos en los que participaron
legislatura_por_alcalde = (
    df_repetidos.groupby(['NOMBRE', 'MUNICIPIO'])['FECHA POSESION']
    .apply(lambda x: sorted(x.dt.year.unique()))
    .reset_index(name='LEGISLATURAS')
)

# Filtrar solo los alcaldes que participaron en más de una legislatura
legislatura_por_alcalde = legislatura_por_alcalde[legislatura_por_alcalde['LEGISLATURAS'].apply(len) > 1]

# Mostrar resultados
legislatura_por_alcalde


,NOMBRE,MUNICIPIO,LEGISLATURAS
0,abad becquer fernando,leganes,"[1983, 1987]"
1,abad benedicto enrique,rinconada (la),"[1987, 1991, 1995, 1999]"
2,abad caballero jose,carabantes,"[1983, 1987]"
3,abad garcia antonio,santa cruz de marchena,"[1991, 1995, 1999]"
4,abad gomez heradio,poblacion de campos,"[1979, 1983, 1991]"
...,...,...,...
21180,zuniga perezlemaur jesus,rozas de madrid (las),"[1983, 1987, 1991]"
21181,zurbano lecumberri jose,cabredo,"[1979, 1983, 1991, 1997]"
21182,zuriaga adrian blas,yesa (la),"[1979, 1991]"
21183,zurriaga manez vicente,olocau,"[1979, 1983]"


<p>2. De repetir mandato. Qué partidos políticos tiveron máis éxito? Qué alcaldes mudaron de partido?</p>

In [5]:
# Contar cuántas veces cada partido tuvo alcaldes reelectos
partidos_exito = df_repetidos.groupby('LISTA').size().reset_index(name='ALCALDES REELECTOS')

# Ordenar de mayor a menor
partidos_exito = partidos_exito.sort_values(by='ALCALDES REELECTOS', ascending=False)

# Mostrar resultados
print(partidos_exito)


# Agrupar por alcalde y municipio para obtener los partidos únicos en los que ha estado
cambios_partido = (
    df_repetidos.groupby(['NOMBRE', 'MUNICIPIO'])['LISTA']
    .apply(lambda x: sorted(x.unique()))
    .reset_index(name='LISTA')
)

# Filtrar solo los alcaldes que han cambiado de partido (más de un partido en la lista)
cambios_partido = cambios_partido[cambios_partido['LISTA'].apply(len) > 1]

# Mostrar resultados
cambios_partido



      LISTA  ALCALDES REELECTOS
64     psoe               17535
55       pp               16600
41      ind                3138
23      ciu                2957
45    otros                1527
..      ...                 ...
73     upsa                   1
37  ganemos                   1
49      pcc                   1
39  guanyem                   1
62      psg                   1

[80 rows x 2 columns]


,NOMBRE,MUNICIPIO,LISTA
2,abad caballero jose,carabantes,"[ap, appdpul]"
4,abad gomez heradio,poblacion de campos,"[appdpul, pp, ucd]"
5,abad pastor eugenio,cabra de mora,"[appdpul, ind]"
7,abad roldan amadeo,congosto de valdavia,"[pp, ucd]"
9,abadia urieta armando,jaca,"[ap, appdpul, pp, ucd]"
...,...,...,...
21173,zuazo jimenez ladislao,castildelgado,"[ap, appdpul, pp]"
21177,zubizarreta lasagabaster xabier,arrasate o mondragon,"[eh, hb]"
21181,zurbano lecumberri jose,cabredo,"[cgest, ind, upn]"
21182,zuriaga adrian blas,yesa (la),"[pp, ucd]"


<p>3. Cantos alcaldes cesaron antes de tempo?</p>

In [10]:
# Convertir las fechas a formato datetime
excel_entero_df['FECHA POSESION'] = pd.to_datetime(excel_entero_df['FECHA POSESION'], errors='coerce', dayfirst=True)
excel_entero_df['FECHA BAJA'] = pd.to_datetime(excel_entero_df['FECHA BAJA'], errors='coerce', dayfirst=True)

# Filtrar alcaldes que cesaron antes del mandato completo 
ceses_anticipados = excel_entero_df[excel_entero_df['DURACION'] < 4]

# Contar cuántos alcaldes cesaron antes de tiempo
total_cesados = ceses_anticipados.shape[0]

# Mostrar el número total de alcaldes cesados antes de tiempo
print(f"Alcaldes que cesaron antes de tiempo: {total_cesados}")

Alcaldes que cesaron antes de tiempo: 5895


5895

<p>4. Cal é o soldo máis alto e de que alcaldes?</p>

In [12]:
ultima_hoja_df = limpiar(excel_entero["Alcaldes 2019-2023"])

ultima_hoja_df

retribuciones2018_df = pd.read_csv("tabula-retribuciones_alcaldes_2021.csv")

retribuciones2018_df

# Convertir la columna 'TOTAL PERCIBIDO' a float
retribuciones2018_df["TOTAL PERCIBIDO"] = (
    retribuciones2018_df["TOTAL PERCIBIDO"]
    .str.replace(".", "", regex=False)  # Eliminar separadores de miles
    .str.replace(",", ".", regex=False)  # Reemplazar coma decimal
    .astype(float)  # Convertir a número
)

# Normalizar las columnas directamente
retribuciones2018_df[["AYUNTAMIENTO", "PROVINCIA"]] = retribuciones2018_df[["AYUNTAMIENTO", "PROVINCIA"]].map(lambda x: unidecode(x.lower().strip()) if isinstance(x, str) else x)


# Hacer el merge con los nombres normalizados
df_merged = ultima_hoja_df.merge(
    retribuciones2018_df,
    left_on=["MUNICIPIO", "PROVINCIA"],
    right_on=["AYUNTAMIENTO", "PROVINCIA"],
    how="inner"  # Mantener solo coincidencias
)

# Seleccionar columnas necesarias
df_ordenado = df_merged[["NOMBRE", "MUNICIPIO", "PROVINCIA", "TOTAL PERCIBIDO"]]

# Ordenar por sueldo de mayor a menor
df_ordenado = df_ordenado.sort_values(by="TOTAL PERCIBIDO", ascending=False)

# Mostrar resultados
df_ordenado


,AYUNTAMIENTO,PROVINCIA,CCAA,RÉGIMEN DEDICACIÓN,TOTAL PERCIBIDO
0,Agost,Alacant/Alicante,Comunitat Valenciana,Exclusiva,"38.902,92"
1,Agres,Alacant/Alicante,Comunitat Valenciana,Parcial,"679,46"
2,Aigües,Alacant/Alicante,Comunitat Valenciana,Parcial,"10.949,48"
3,Albatera,Alacant/Alicante,Comunitat Valenciana,Sin dedicación,"0,00"
4,Alcalalí,Alacant/Alicante,Comunitat Valenciana,Parcial,"23.999,64"
...,...,...,...,...,...
7019,"Vilueña, La",Zaragoza,Aragón,Sin dedicación,"0,00"
7020,Vistabella,Zaragoza,Aragón,Sin dedicación,"0,00"
7021,"Zaida, La",Zaragoza,Aragón,Sin dedicación,"0,00"
7022,Zaragoza,Zaragoza,Aragón,Exclusiva,"81.388,72"


<p>5. Relaciona os soldos altos e baixos coa "probabilidade" de repetir mandato.</p>

<p>6. Outra información interesante.</p>
